In [19]:
# avoid recursion limit errors
import sys
sys.setrecursionlimit(10000)
import random
import heapq
import math
import sys
import time
from collections import defaultdict, deque, Counter
from itertools import combinations
from IPython.display import clear_output

import chess
import chess.engine

import numpy as np
import pandas as pd

# Turn Based Games  : Chess

In the turn based games there are at two agents that play competitively.
In this project have been implemented different classes in order to represent the game, the player, the state and the heurisitcs.

### Abstract Class Game

This class can be used to implement a chess game instance

In [20]:
    """actions: Return a collection of the allowed moves of the current state"""
    def actions(self, state):
        raise NotImplementedError
        
    """result:Return the state that results from making a move from a state."""
    def result(self, state, move):
        raise NotImplementedError
        
    """is_terminal: Return True if this is a final state for the game."""
    def is_terminal(self, state):
        return not self.actions(state)
    
    """get_player: Returns the current player"""    
    def get_player(self, state):
        raise NotImplementedError
        
    """get_turn: Returns the turn of the player that has to make a move"""    
    def get_turn(self, state):
        raise NotImplementedError

# Game Class

In [21]:
class Game():
    """A game is similar to a problem, but it has a terminal test instead of 
    a goal test, and a utility for each terminal state. To create a game, 
    subclass this class and implement `actions`, `result`, `is_terminal`, 
    and `h`."""

    def actions(self, state):
        """Return a collection of the allowable moves from this state."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def is_terminal(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)
        
    def get_player(self, state):
        """Returns the player whose turn it is"""
        raise NotImplementedError
        
    def get_turn(self, state):
        """Returns the turn of the player that has to make a move"""
        raise NotImplementedError

# Chess Class

this class will be used to instantiate a new chess game board. The board is represented by using Forsyth–Edwards Notation (FEN) that is a standard notation for describing a particular board position of a chess game.

In [22]:
#
# Chess game
#
class Chess(Game):
    """Instantiate a chess game usind the python-chess library"""
    
    def __init__(self):
        self.initial = chess.Board().fen()
        
    def actions(self, state):
        board = self.get_board(state)
        return board.legal_moves
    
    def result(self, state, action):
        board = self.get_board(state)
        board.push(action)
        return board.fen()

    def undo_move(self, state):
        board = self.get_board(state)
        board.pop()
    
    def is_terminal(self, state):
        board = self.get_board(state)
        outcome = board.outcome();
        if (outcome != None):
            return True;
        else:
            return False
        
    def display(self, state):
        board = self.get_board(state)
        display(board)
        
    def get_board(self, state):
        """Given a FEN state, returns the actual board"""
        return chess.Board(state)  
    
    def get_player(self, state):
        """Returns the player whose turn it is"""
        board = self.get_board(state)
        return 'White' if board.turn else 'Black'
    
    def get_turn(self, state):
        return self.get_board(state).turn
    
    def get_move(self, move_str):
        """Transform a string to a Move object"""
        move = None
        if len(move_str) == 4 or len(move_str) == 5:
            try:
                move = chess.Move.from_uci(move_str)
            except ValueError:
                move = None
                # do nothing
        return move

# Abstract Class Heuristic

In [23]:
class Heuristics():
    """This abstract class has to be implemented to define the heuristic
       evaluation for a particular game. All logic related to the evaluation
       should be defined here"""
     
    def h1(self, state):
        """This is the first heuristic. There should be atleast one"""
        raise NotImplementedError

In [24]:
class ChessHeuristics(Heuristics):
    """Class where all the chess heuristic are defined"""
    
    def __init__(self):
        self.weights = np.array([1.2, 1.2, 1.2, 0.9, 0.9, 0.6])
        self.pieces_value = {
            'p': 100, 'P': 100,
            'n': 280, 'N': 280,
            'b': 320, 'B': 320,
            'r': 479, 'R': 479,
            'q': 929, 'Q': 929,
            'k': 60000, 'K': 60000,
        }
        self.squares_value = {
            'P': (  0,   0,   0,   0,   0,   0,   0,   0,
                    -31,   8,  -7, -37, -36, -14,   3, -31,
                    -22,   9,   5, -11, -10,  -2,   3, -19,
                    -26,   3,  10,   9,   6,   1,   0, -23,
                    -17,  16,  -2,  15,  14,   0,  15, -13,
                    7,  29,  21,  44,  40,  31,  44,   7,
                    78,  83,  86,  73, 102,  82,  85,  90,
                    0,   0,   0,   0,   0,   0,   0,   0),

            'N': (  -74, -23, -26, -24, -19, -35, -22, -69,
                    -23, -15,   2,   0,   2,   0, -23, -20,
                    -18,  10,  13,  22,  18,  15,  11, -14,
                    -1,   5,  31,  21,  22,  35,   2,   0,
                    24,  24,  45,  37,  33,  41,  25,  17,
                    10,  67,   1,  74,  73,  27,  62,  -2,
                    -3,  -6, 100, -36,   4,  62,  -4, -14,
                    -66, -53, -75, -75, -10, -55, -58, -70),

            'B': (  -7,   2, -15, -12, -14, -15, -10, -10,
                    19,  20,  11,   6,   7,   6,  20,  16,
                    14,  25,  24,  15,   8,  25,  20,  15,
                    13,  10,  17,  23,  17,  16,   0,   7,
                    25,  17,  20,  34,  26,  25,  15,  10,
                    -9,  39, -32,  41,  52, -10,  28, -14,
                    -11,  20,  35, -42, -39,  31,   2, -22,
                    -59, -78, -82, -76, -23,-107, -37, -50),

            'R': (  -30, -24, -18,   5,  -2, -18, -31, -32,
                    -53, -38, -31, -26, -29, -43, -44, -53,
                    -42, -28, -42, -25, -25, -35, -26, -46,
                    -28, -35, -16, -21, -13, -29, -46, -30,
                    0,   5,  16,  13,  18,  -4,  -9,  -6,
                    19,  35,  28,  33,  45,  27,  25,  15,
                    55,  29,  56,  67,  55,  62,  34,  60,
                    35,  29,  33,   4,  37,  33,  56,  50),

            'Q': (  -39, -30, -31, -13, -31, -36, -34, -42,
                    -36, -18,   0, -19, -15, -15, -21, -38,
                    -30,  -6, -13, -11, -16, -11, -16, -27,
                    -14, -15,  -2,  -5,  -1, -10, -20, -22,
                    1, -16,  22,  17,  25,  20, -13,  -6,
                    -2,  43,  32,  60,  72,  63,  43,   2,
                    14,  32,  60, -10,  20,  76,  57,  24,
                    6,   1,  -8,-104,  69,  24,  88,  26),

            'K': (  17,  30,  -3, -14,   6,  -1,  40,  18,
                    -4,   3, -14, -50, -57, -18,  13,   4,
                    -47, -42, -43, -79, -64, -32, -29, -32,
                    -55, -43, -52, -28, -51, -47,  -8, -50,
                    -55,  50,  11,  -4, -19,  13,   0, -49,
                    -62,  12, -57,  44, -67,  28,  37, -31,
                    -32,  10,  55,  56,  56,  55,  10,   3,
                    4,  54,  47, -99, -99,  60,  83, -62),

            'p': (   0,   0,   0,   0,   0,   0,   0,   0,
                    78,  83,  86,  73, 102,  82,  85,  90,
                     7,  29,  21,  44,  40,  31,  44,   7,
                   -17,  16,  -2,  15,  14,   0,  15, -13,
                   -26,   3,  10,   9,   6,   1,   0, -23,
                   -22,   9,   5, -11, -10,  -2,   3, -19,
                   -31,   8,  -7, -37, -36, -14,   3, -31,
                     0,   0,   0,   0,   0,   0,   0,   0),

            'n': ( -66, -53, -75, -75, -10, -55, -58, -70,
                    -3,  -6, 100, -36,   4,  62,  -4, -14,
                    10,  67,   1,  74,  73,  27,  62,  -2,
                    24,  24,  45,  37,  33,  41,  25,  17,
                    -1,   5,  31,  21,  22,  35,   2,   0,
                   -18,  10,  13,  22,  18,  15,  11, -14,
                   -23, -15,   2,   0,   2,   0, -23, -20,
                   -74, -23, -26, -24, -19, -35, -22, -69),

            'b': ( -59, -78, -82, -76, -23,-107, -37, -50,
                   -11,  20,  35, -42, -39,  31,   2, -22,
                    -9,  39, -32,  41,  52, -10,  28, -14,
                    25,  17,  20,  34,  26,  25,  15,  10,
                    13,  10,  17,  23,  17,  16,   0,   7,
                    14,  25,  24,  15,   8,  25,  20,  15,
                    19,  20,  11,   6,   7,   6,  20,  16,
                    -7,   2, -15, -12, -14, -15, -10, -10),

            'r': (  35,  29,  33,   4,  37,  33,  56,  50,
                    55,  29,  56,  67,  55,  62,  34,  60,
                    19,  35,  28,  33,  45,  27,  25,  15,
                     0,   5,  16,  13,  18,  -4,  -9,  -6,
                   -28, -35, -16, -21, -13, -29, -46, -30,
                   -42, -28, -42, -25, -25, -35, -26, -46,
                   -53, -38, -31, -26, -29, -43, -44, -53,
                   -30, -24, -18,   5,  -2, -18, -31, -32),

            'q': (   6,   1,  -8,-104,  69,  24,  88,  26,
                    14,  32,  60, -10,  20,  76,  57,  24,
                    -2,  43,  32,  60,  72,  63,  43,   2,
                     1, -16,  22,  17,  25,  20, -13,  -6,
                   -14, -15,  -2,  -5,  -1, -10, -20, -22,
                   -30,  -6, -13, -11, -16, -11, -16, -27,
                   -36, -18,   0, -19, -15, -15, -21, -38,
                   -39, -30, -31, -13, -31, -36, -34, -42),

            'k': (   4,  54,  47, -99, -99,  60,  83, -62,
                   -32,  10,  55,  56,  56,  55,  10,   3,
                   -62,  12, -57,  44, -67,  28,  37, -31,
                   -55,  50,  11,  -4, -19,  13,   0, -49,
                   -55, -43, -52, -28, -51, -47,  -8, -50,
                   -47, -42, -43, -79, -64, -32, -29, -32,
                    -4,   3, -14, -50, -57, -18,  13,   4,
                    17,  30,  -3, -14,   6,  -1,  40,  18)}
        
        
    def h1(self, board):
        """Random pick; given a state, this heuristic returns a random number
        that randomize the action pick if all evaluations are the same"""
        bonus = round(random.random(), 2)*10
        return bonus
    
    def h2(self, board):
        """Checkmate heuristic; if state is checkmate receives big reward"""
        value = 0;
        if (board.is_checkmate()):
            value += 9999
        return value
    
    def h3(self, board):
        """Check heuristic; it rewards a little the states with a check on the king"""
        
        value = 0;
        if (board.is_check()):
            value += 25
        return value
    
    def h4(self, board):
        """White pieces evaluation; it assign values based on the number and quality of the pieces"""
        value_white = 0
        
        pieces = board.piece_map()
        
        for p in pieces:
            if(str(pieces[p]).isupper()):
                value_white += self.pieces_value[str(pieces[p])]
                       
        return value_white if not board.turn else - value_white
                
    def h5(self, board):
        """Black pieces evaluation; it assign values based on the number and quality of the pieces"""
        value_black = 0
        
        pieces = board.piece_map()
        
        for p in pieces:
            if(not str(pieces[p]).isupper()):
                value_black += self.pieces_value[str(pieces[p])]
     
        return value_black if board.turn else - value_black
    
    def h6(self, board):
        """Evaluate goodness of pieces position on the board"""
        value_black = 0
        value_white = 0
        
        pieces = board.piece_map()
        
        for p in pieces:
            if(not str(pieces[p]).isupper()):
                value_black += self.squares_value[str(pieces[p])][p]
            else:
                value_white += self.squares_value[str(pieces[p])][p]
        
        return value_white if not board.turn else value_black  
    
    def h(self, game, state): 
        """Linear combination of the heuristics"""
        b = game.get_board(state)
        h = np.array([self.h1(b), self.h2(b), self.h3(b), self.h4(b), self.h5(b), self.h6(b)])
        lc = self.weights * h
        
        return round(lc.sum(), 4)
    
    def h_as_arr(self, game, state):
        b = game.get_board(state)
        h = np.array([self.h1(b), self.h2(b), self.h3(b), self.h4(b), self.h5(b), self.h6(b)])
        return np.around(self.weights * h, decimals=4)

# Alpha-Beta Pruning

In [25]:
#
# Alpha-Beta Pruning 
#

# make the depth specifiable
def alpha_beta_algorithm_depth(depth=3):
    return lambda game, state, heuristics: alpha_beta_algorithm(game, state, heuristics, depth = depth)

# init of the algorithm
def alpha_beta_algorithm(game, state, heuristics, depth = 3):
    
    def alpha_beta(game, state, depth, alpha, beta, maximizing_player):
        if depth == 0 or game.is_terminal(state):
            return heuristics.h(game, state)

        if maximizing_player:
            max_eval = -np.inf
            for move in game.actions(state):
                new_state = game.result(state, move)
                eval = alpha_beta(game, new_state, depth - 1, alpha, beta, False)
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return max_eval
        else:
            min_eval = np.inf
            for move in game.actions(state):
                new_state = game.result(state, move)
                eval = alpha_beta(game, new_state, depth - 1, alpha, beta, True)
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return min_eval


    # init
    max_move = None
    max_eval = -np.inf
    
    for move in game.actions(state):
        new_state = game.result(state, move)
        eval = alpha_beta(game, new_state, depth - 1, -np.inf, np.inf, False)
        if eval > max_eval:
            max_eval = eval
            max_move = move
            
    return max_eval, max_move

In [ ]:
def alpha_beta_pruning (game,heuristics, state, depth, alpha, beta, max_player):
    if depth == 0 or game.is_terminal(state)
        return heuristics.h(game,state)
    
    if max_player:
        max_ev = -math.inf
        for move in game.actions(state):
            new_state = game.result(state,move)
            ev = alpha_beta_pruning(game, new_state, depth-1, alpha, beta, False)
            max_ev = max(max_ev, ev)
            alpha = (alpha, ev)
            if beta<= alpha:
                break
        return max ev;
    if not max_player:
        min_ev = math.inf
        for move in game.actions(state):
            new_state = game.result(state, move)
            ev = ev = alpha_beta_pruning(game, new_state, depth-1, alpha, beta, True)
            min_ev = min(min_ev, ev)
            beta = min(beta, ev)
            if beta <= alpha
            break
        return min_ev

def get_best_move(game, state,depth, -math.inf, math.inf, not max_player):
    max_move = None
    max_eval = -np.inf
    
    for move in game.actions(state):
        new_state = game.result(state, move)
        ev = alpha_beta(game,heuristics, new_state, depth - 1, -np.inf, np.inf, False)
        if ev > max_eval:
            max_eval = ev
            max_move = move
            
    return max_eval, max_move

# Player

In [26]:
class Player():
    def __init__(self, name: str, strategy, heuristics: Heuristics):
        self.name = name
        self.strategy = strategy
        self.heuristics = heuristics
        
    def get_move(self, game: Game, state):
        return self.strategy(game, state, self.heuristics)

In [27]:
#
# Different playing strategies
#
def greedy_strategy(game: Game, state, heuristics: Heuristics):
    actions = list(game.actions(state))
    best_a = actions[0]
    best_v = heuristics.h(game, game.result(state, best_a))
    
    for action in actions:
        new_state = game.result(state, action)
        v = heuristics.h(game, new_state)
        if v > best_v:
            best_v = v
            best_a = action
            
    return best_v, best_a

def play_yourself_strategy(game: Game, state, heuristics: Heuristics):
    """Make a move by querying standard input."""
    print("Available moves: {}".format(game.actions(state)))
    print("")
    move = None


    while True:
        move_str = input("Your move? ")

        if game.get_move(move_str) in game.actions(state):
            break
        else:
            print("Please type a valid move")
            
    move = game.get_move(move_str)   
    
    return None, move

def random_strategy(game: Game, state, heuristics: Heuristics):
    return None, random.choice(list(game.actions(state)))

In [14]:
#
# Different players
#
query_player = Player("QueryPlayer", play_yourself_strategy, ChessHeuristics())
alpha_beta_player_depth3 = Player("AlphaBetaPlayerDepth3", alpha_beta_algorithm_depth(3), ChessHeuristics())
random_player = Player("RandomPlayer", random_strategy, ChessHeuristics())
greedy_player = Player("GreedyPlayer", greedy_strategy, ChessHeuristics())

alpha_beta_player_depth2 = Player("AlphaBetaPlayerDepth2", alpha_beta_algorithm_depth(2), ChessHeuristics())
alpha_beta_player_depth1 = Player("AlphaBetaPlayerDepth1", alpha_beta_algorithm_depth(1), ChessHeuristics())

Play

In [28]:
#
# Infrastructure for see the agents playing
#
def play_game_live(game, players: dict, verbose=False):
    """Play a turn-taking game. `strategies` is a {player_name: function} dict,
    where function(state, game) is used to get the player's move."""
    state = game.initial
    game.display(state)
    while not game.is_terminal(state):

        player_name = game.get_player(state)
        print("Player <{}> is thinking...".format(player_name))
        v, move = players[player_name].get_move(game, state)
        print (v)
        print(move)
        time.sleep(3)
        
        state = game.result(state, move)
        if verbose: 
            clear_output(wait=True)
            game.display(state)
            print("Player <{}> choosed the move: {} with a value of {}".format(player_name, move, v))
            #time.sleep(1)

    return state

In [29]:
#def play_game()

In [30]:
#
# Infrastructure to collect data from a number of chess games
#
def generate_chess_row(players: dict, final_board, time, moves_number):
    winner = final_board.outcome().winner
    winner_player = None
    if (winner == True):
        winner_player = 'White'
    if (winner == False):
        winner_player = 'Black'
    
    row = {'White': players['White'].name,
           'Black': players['Black'].name,
           'final_state': final_board.fen(),
           'winner': winner_player,
           'termination': final_board.outcome().termination,
           'game_time(s)': f'{time:.2f}',
           'total_moves': moves_number}
    return row

def play_chess_games(game, players: dict, games_number=50, update=False, verbose=True):
    """Play a turn-taking game. `strategies` is a {player_name: function} dict,
    where function(state, game) is used to get the player's move."""
    
    outcome_list = []
    for i in range(games_number):
        
        moves_number = 0
        t0 = time.time()
        state = game.initial
        if (verbose):
            print(f'Game {i} started')
        
        while not game.is_terminal(state):

            player_name = game.get_player(state)
            v, move = players[player_name].get_move(game, state)
            state = game.result(state, move)
            moves_number += 1

        final_board = game.get_board(state)
        
        t1 = time.time() - t0
        outcome_row = generate_chess_row(players, final_board, t1, moves_number)
        outcome_list.append(outcome_row)

        if (verbose):
            print(f'Game {i} ended in {t1:.2f}s: {final_board.outcome()}')
            print('\n')
        
        if (update):
            update_dataset()
    
    
    return outcome_list

In [34]:
result = play_game_live(Chess(), dict(White=greedy_player, Black=random_player), verbose=True)

KeyboardInterrupt: 

In [18]:
result = play_chess_games(Chess(), dict(White=greedy_player, Black=alpha_beta_player_depth3), 1, verbose=True)

Game 0 started
Game 0 ended in 440.35s: Outcome(termination=<Termination.CHECKMATE: 1>, winner=False)




In [ ]:
print(result)

In [ ]:
df = pd.DataFrame(result)
df